# Using Container classes

`ctapipe.core.Container` is the base class for all event-wise data classes in ctapipe. It works like a object-relational mapper, in that it defines a set of `Fields` along with their metadata (description, unit, default), which can be later translated automatially into an output table using a `ctapipe.io.TableWriter`.

In [ ]:
from ctapipe.core import Container, Field, Map
import numpy as np
from astropy import units as u

Let's define a few example containers with some dummy fields in them:

In [ ]:
class SubContainer(Container):
    junk = Field("nothing","Some junk")
    value = Field(0.0, "some value", unit=u.deg)

class EventContainer(Container):
    event_id = Field(-1,"event id number")
    tels_with_data = Field([], "list of telescopes with data")
    sub = Field(SubContainer(), "stuff")  # a sub-container in the hierarchy

    # for dicts of sub-containers, use Map instead 
    # of a dict() as the default value to support serialization
    tel = Field(Map(), "telescopes")  


## Basic features

In [ ]:
ev = EventContainer()

Check that default values are automatically filled in

In [ ]:
print(ev.event_id)
print(ev.tel.keys())
print(ev.tel)

print the json representation

In [ ]:
print(ev)

values can be set as normal for a class:

In [ ]:
ev.event_id = 100
ev.event_id

In [ ]:
ev.as_dict()  # by default only shows the bare items, not sub-containers (See later)

In [ ]:
ev.as_dict(recursive=True)

Now, let's define a sub-container that we can add per telescope:

In [ ]:
class TelContainer(Container):
    tel_id = Field(-1, "telescope ID number")
    image = Field(np.zeros(10), "camera pixel data")



and we can add a few of these to the parent container inside the tel dict:

In [ ]:
ev.tel[10] = TelContainer()
ev.tel[5] = TelContainer()
ev.tel[42] = TelContainer()

In [ ]:
ev.tel

## Converion to dictionaries

In [ ]:
ev.as_dict()

In [ ]:
ev.as_dict(recursive=True, flatten=False)

for serialization to a table, we can even flatten the output into a single set of columns

In [ ]:
ev.as_dict(recursive=True, flatten=True)

## Setting and clearing values

In [ ]:
ev.tel[5].image[:] = 9
print(ev)

In [ ]:
ev.reset()
ev.as_dict(recursive=True, flatten=True)

## look at a pre-defined Container

In [ ]:
from ctapipe.containers import SimulatedShowerContainer

In [ ]:
shower = SimulatedShowerContainer()

In [ ]:
shower